Ce programme permet de lire les éléments contenu dans le tableau d'un fichier pdf et de les enregistrer dans un fichier excel

In [6]:
import PyPDF2
import openpyxl
from openpyxl import Workbook
import pandas as pd

In [7]:
import pdfplumber

def extract_table_data(pdf_path, page_number):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number]
        table = page.extract_tables()[0]

        headers = table[0]
        data = table[1:]

        extracted_data = []
        for row in data:
            extracted_row = dict(zip(headers, row))
            extracted_data.append(extracted_row)

        return extracted_data


In [8]:
import glob 

files = glob.glob(r"./pdf/*.pdf")

['./pdf\\FACR R 04-03-2020.pdf',
 './pdf\\FACT ACCOUPLEMENT 16-09-2020.pdf',
 './pdf\\FACT AMPOULE 10-07-2020.pdf',
 './pdf\\FACT BATTERIE 19-10-2020.pdf',
 './pdf\\FACT CHIFFON 11-11-2020.pdf',
 './pdf\\FACT CHIFFON 29-12-2020.pdf',
 './pdf\\FACT COFRET 29-01-2020.pdf',
 './pdf\\FACT COURROIE 06-11-2020.pdf',
 './pdf\\FACT DES 100 FILTRE.pdf',
 './pdf\\FACT DES ACCOUPLEMENTS.pdf',
 './pdf\\FACT DES CHIFFON COTON 18-02-2022.pdf',
 './pdf\\FACT DES FILTRES 11-01-2022.pdf',
 './pdf\\FACT DES FILTRES 20-09-2022.pdf',
 './pdf\\FACT DES FILTRES 24-05-2023.pdf',
 './pdf\\FACT DES FILTRES 29-01-2023.pdf',
 './pdf\\FACT DES FILTRES.pdf',
 './pdf\\FACT DES PREFILTRES 27-05-2022.pdf',
 './pdf\\FACT DURITE 31-01-2020.pdf',
 './pdf\\FACT EAU DISTILLEE 19-05-2020.pdf',
 './pdf\\FACT FILTRE 15-05-2020.pdf',
 './pdf\\FACT FORET 17-12-2019.pdf',
 './pdf\\FACT GRAIISSE.pdf',
 './pdf\\FACT JOINT 06-07-2020.pdf',
 './pdf\\FACT JOINT 18-11-2020.pdf',
 './pdf\\FACT JOINT SPI 17-02-2020.pdf',
 './pdf\\FACT 

In [10]:
ln =1
fl = Workbook()
sheet = fl.active

sheet["A1"] = "Code"
sheet["B1"] = "Date"
sheet["C1"] = "Designation"
sheet["D1"] = "Reference"
sheet["E1"] = "marque"
sheet["F1"] = "origine"
sheet["G1"] = "QTE"
sheet["H1"] = "Prix HT"
sheet["I1"] = "Montant HT"

for filename in files :
    
    with open(f"{filename}", 'rb') as pdf:
        reader = PyPDF2.PdfReader(pdf)
        texte_extrait = reader.pages[0].extract_text()
        numero_devis = texte_extrait[texte_extrait.find('devis :')+7 : texte_extrait.find('\n', texte_extrait.find('devis :'))]
        date = texte_extrait[texte_extrait.find('Date :')+6 : texte_extrait.find('\n', texte_extrait.find('Date :'))]
    data = {'code':numero_devis,'date':date}
        
    pdf_path = f"{filename}"
    pg_number = 0
    table_data = extract_table_data(pdf_path, pg_number)

    ln += 1

    for prod in table_data[:-1]:
        sheet[f"A{ln}"] = data["code"]
        sheet[f"B{ln}"] = data["date"]
        sheet[f"C{ln}"] = prod["Désignation"]
        sheet[f"D{ln}"] = prod["Référence"]
        sheet[f"E{ln}"] = prod["marque"]
        sheet[f"F{ln}"] = prod["origine"]
        sheet[f"G{ln}"] = prod["QTE"]
        sheet[f"H{ln}"] = prod["Prix HT"]
        if "Montan HT" in prod:
            sheet[f"I{ln}"] = prod["Montan HT"]
        elif "Montan\nHT" in prod:
            sheet[f"I{ln}"] = prod["Montan\nHT"]
        ln += 1

fl.save("excel/factures.xlsx")